### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

### Observações

#### Sobre como cria tabela CSV com limite documentos por tópico

### Código

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K4200 (CNMeM is disabled, cuDNN 5103)
/usr/local/lib/python3.5/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is depreca

In [2]:
outputs = "../outputs/"

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs")
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')
        
        text =  row[1]  
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)
        
        #if count == 3: break #amostra de apenas 3 linhas. No computador do Renato, não usar o break.

100 % done


In [4]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
            
stopwords = list(set(stopwords))

In [6]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in texts]

CPU times: user 2min 13s, sys: 36 ms, total: 2min 14s
Wall time: 2min 14s


In [7]:
print(len(texts[0]))

125


In [8]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [9]:
len(corpus)

10225

In [10]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

CPU times: user 55min 10s, sys: 4h 20min 21s, total: 5h 15min 31s
Wall time: 19min 17s


In [11]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.016*argentina + 0.015*brasil + 0.009*buenos + 0.008*paraguay + 0.007*aires + 0.007*corpus + 0.007*itaipu + 0.005*gobierno + 0.005*acuerdo + 0.005*argentino + 0.004*política + 0.004*recebido + 0.004*telegrama + 0.003*parana + 0.003*relaciones + 0.003*rio + 0.003*cuenca + 0.003*claudio + 0.003*artigo + 0.003*si'),
 (1,
  '0.009*nacional + 0.008*desenvolvimento + 0.008*projeto + 0.008*cooperação + 0.006*recursos + 0.005*atividades + 0.005*projetos + 0.005*programa + 0.005*brasileira + 0.005*técnica + 0.005*trabalho + 0.004*ministério + 0.004*estudos + 0.004*brasil + 0.003*tecnologia + 0.003*participação + 0.003*produção + 0.003*setor + 0.003*industrial + 0.003*planejamento'),
 (2,
  '0.017*artigo + 0.011*art + 0.008*acordo + 0.007*lei + 0.006*federal + 0.005*presente + 0.004*república + 0.004*parágrafo + 0.004*ministério + 0.004*brasil + 0.004*governo + 0.004*vigor + 0.004*prazo + 0.004*decreto + 0.003*contratantes + 0.003*direito + 0.003*cada + 0.003*brasileiro + 0.003*data + 0

In [12]:
lda.show_topics(num_topics=-1)

[(0,
  '0.016*argentina + 0.015*brasil + 0.009*buenos + 0.008*paraguay + 0.007*aires + 0.007*corpus + 0.007*itaipu + 0.005*gobierno + 0.005*acuerdo + 0.005*argentino'),
 (1,
  '0.009*nacional + 0.008*desenvolvimento + 0.008*projeto + 0.008*cooperação + 0.006*recursos + 0.005*atividades + 0.005*projetos + 0.005*programa + 0.005*brasileira + 0.005*técnica'),
 (2,
  '0.017*artigo + 0.011*art + 0.008*acordo + 0.007*lei + 0.006*federal + 0.005*presente + 0.004*república + 0.004*parágrafo + 0.004*ministério + 0.004*brasil'),
 (3,
  '0.056*nigéria + 0.056*lagos + 0.038*festival + 0.023*nigeria + 0.018*trofeos + 0.015*black + 0.013*1971-1975 + 0.010*difamação + 0.009*culture + 0.007*art'),
 (4,
  '0.005*ar + 0.004*oe + 0.003*rr + 0.003*pr + 0.003*ro + 0.003*mw + 0.003*lu + 0.003*ue + 0.003*po + 0.003*cc'),
 (5,
  '0.122*costa + 0.051*marfim + 0.019*abidjan + 0.013*patrulha + 0.010*afetuosamente + 0.010*huch + 0.008*interêsses + 0.008*maçãs + 0.006*leitorado + 0.006*iica'),
 (6,
  '0.066*unidos

In [13]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[11]

(11,
 [('states', 0.0074589829602145063),
  ('united', 0.0068090491537654545),
  ('world', 0.0059461952804586542),
  ('government', 0.00521875270207885),
  ('brazil', 0.0051613087787754945),
  ('international', 0.0042985115175983362),
  ('nations', 0.0040259380389478334),
  ('rights', 0.0039167402258627566),
  ('us', 0.0038559238414734125),
  ('economic', 0.0038432327696181811),
  ('foreign', 0.0036504277029891616),
  ('national', 0.0034531943393815764),
  ('human', 0.0034282246160120761),
  ('time', 0.0033385002994869759),
  ('american', 0.0032899681798039555),
  ('minister', 0.0032795492332954863),
  ('state', 0.0032116987463198261),
  ('country', 0.0032005844340895965),
  ('people', 0.0031789794296034493),
  ('policy', 0.0031636817641271667)])

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

In [14]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc ORDER BY topic_id ASC, topic_score DESC")
        
        #if count == 10: break #amostra de apenas 10 linhas

········
········
[(0, 0.27933381892473491), (30, 0.20716241912850872), (39, 0.14752094075911487), (41, 0.077894044716182984), (44, 0.21907012558755429), (47, 0.059018650883904164)]
rb_1974.04.17_doc_I-66 47 0.0587751828461
[(10, 0.20200677234404288), (20, 0.45083066429203728), (33, 0.1770588235294116), (35, 0.032569735520493394), (36, 0.11106341607871989)]
onu_1974.03.12_doc_II-49 36 0.111175521086
[(4, 0.083463396857084857), (12, 0.87653660314291582)]
onu_1974.03.12_doc_V-2 12 0.87654511474
[(6, 0.041408255063709501), (7, 0.78549704142487153), (9, 0.017719739762147923), (13, 0.014575564476687376), (35, 0.020534457628025218), (39, 0.041748330353037731), (45, 0.028593785496366143), (47, 0.036727898015829173)]
be_1974.04.16_doc_VI-18 47 0.0367273315578
[(2, 0.16317561839252898), (6, 0.17851464320925439), (13, 0.32665854474960965), (17, 0.058718720076900544), (18, 0.071951507743109086), (22, 0.016425581474903809), (35, 0.045767510004392234), (39, 0.13336851951059117)]
be_1977.06.01_doc_I

#### Visualizando os tópicos com o PyLDAvis

In [15]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
13     16.161759        1       1  0.168561 -0.232080
11      8.468647        1       2 -0.048020  0.289027
39      6.990304        1       3  0.220771 -0.058313
12      6.472678        1       4 -0.006216  0.311284
19      5.497295        1       5  0.231112 -0.112345
2       4.826306        1       6  0.211915 -0.134636
1       4.555991        1       7  0.212910 -0.159563
35      4.258828        1       8  0.159646 -0.118893
7       4.021202        1       9  0.238860 -0.105364
44      3.936372        1      10  0.208390 -0.073404
18      3.926672        1      11  0.251531 -0.039219
4       3.894277        1      12  0.054200  0.095807
46      3.409106        1      13  0.088158 -0.056345
8       3.236457        1      14  0.223907 -0.101873
22      2.423917        1      15  0.159877 -0.081070
0       2.292490        1      16  0.115160  0.015783
36      1.607729        1      17  0.146138  0.183215
28      1.361832        1      18  0.173894 -0.086103
41      1.296436        1      19  0.104059  0.025999
6       1.260484        1      20  0.138602  0.017923
47      1.176861        1      21  0.117988  0.151443
23      1.106708        1      22 -0.036240  0.199607
10      1.000814        1      23  0.119053  0.158153
43      0.983368        1      24  0.079501  0.087428
37      0.810321        1      25  0.118374  0.107189
16      0.807856        1      26  0.100280  0.045119
20      0.771890        1      27  0.138056  0.002911
29      0.657622        1      28  0.133102  0.083937
17      0.581088        1      29  0.045159  0.025422
30      0.397944        1      30  0.018064  0.161732
9       0.269145        1      31 -0.043548  0.061995
48      0.200646        1      32 -0.108546  0.021542
14      0.199776        1      33 -0.040805 -0.009641
34      0.197353        1      34 -0.098753  0.059210
15      0.161078        1      35 -0.168785  0.019950
26      0.121758        1      36 -0.181511  0.013224
42      0.109850        1      37 -0.183723 -0.004823
32      0.105549        1      38 -0.197150 -0.028208
25      0.088081        1      39 -0.165951 -0.024794
5       0.078482        1      40 -0.212487 -0.025135
40      0.064301        1      41 -0.208567 -0.036979
38      0.056747        1      42 -0.226571 -0.037494
31      0.037083        1      43 -0.249930 -0.065369
3       0.029085        1      44 -0.252234 -0.070831
45      0.022504        1      45 -0.244676 -0.060347
24      0.018316        1      46 -0.258040 -0.076305
33      0.014813        1      47 -0.260674 -0.079907
27      0.011919        1      48 -0.262444 -0.083248
21      0.011552        1      49 -0.258462 -0.088683
49      0.008707        1      50 -0.263934 -0.086928, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
76709  Default  22127.000000       presidente  22127.000000  30.0000  30.0000
49591  Default  13867.000000        telegrama  13867.000000  29.0000  29.0000
31533  Default  50956.000000           brasil  50956.000000  28.0000  28.0000
22458  Default  22917.000000         ministro  22917.000000  27.0000  27.0000
93271  Default  19251.000000           senhor  19251.000000  26.0000  26.0000
88724  Default   7648.000000          nuclear   7648.000000  25.0000  25.0000
87856  Default   9462.000000         silveira   9462.000000  24.0000  24.0000
1088   Default  24264.000000         relações  24264.000000  23.0000  23.0000
88072  Default   8863.000000           unidos   8863.000000  22.0000  22.0000
98011  Default  17295.000000       exteriores  17295.000000  21.0000  21.0000
27812  Default  11705.000000       embaixador  11705.000000  20.0000  20.0000
22967  Default  12867.000000       excelência  12867.000000  19.0000  19.0000
68996  Default   6119.000000          azeredo   6119.0000

In [16]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))